### Imports

In [4]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper

import os

In [3]:
os.environ['OPENAI_API_KEY'] = ''

# Tiny Hacking Assignment!

## Generate SQL / Pandas code from natural language
There are 2 datasets in this repo (/data folder). Your goal is to transform the following questions into SQL queries (or python pandas code), using agents. 
ReAct prompting might help you achieve a more optimal solution.

1. (easy) Write an SQL query to retrieve the full names (first name and last name) and email addresses of all customers who have registered in the database since January 1, 2023.
2. Calculate the lifetime value of each customer. Lifetime value is defined as the total sum of orders placed by each customer from their first order to the present. The query should return the customer's first name, last name, and their lifetime value, sorted by lifetime value in descending order.
3. Construct a SQL query to identify customers who placed orders in their first month but did not place any orders in the subsequent month. The query should return the CustomerID, first name, last name, and the date of their last order. Additionally, provide a count of how many orders were placed in the first month of their activity.
4. Devise a SQL query that lists each customer along with their most recent order details. The output should include the customer's first name, last name, and the details of their most recent order (OrderID, OrderDate, TotalAmount, and OrderStatus). Additionally, calculate the number of days that have elapsed since their last order. Sort the results by the elapsed days in ascending order.


Here is a description of the datasets:
### Table 1: customers
This table stores information about the customers who purchase items from the e-commerce store.

Columns:
- CustomerID (INT): A unique identifier for each customer.
- FirstName (VARCHAR): The first name of the customer.
- LastName (VARCHAR): The last name of the customer.
- Email (VARCHAR): The email address of the customer.
- PhoneNumber (VARCHAR, optional): The phone number of the customer, which can be null if not provided.
- CreateDate (DATETIME): The date and time when the customer was added to the database.

### Table 2: orders
This table records details about orders placed by customers.

Columns:
- OrderID (INT): A unique identifier for each order.
- CustomerID (INT): A reference to CustomerID from the Customers table, linking each order to a specific customer.
- OrderDate (DATETIME): The date and time when the order was placed.
- ShippingAddress (VARCHAR): The address where the order is to be shipped.
- TotalAmount (DECIMAL): The total amount of the order in dollars.
- OrderStatus (VARCHAR): The status of the order (e.g., "Pending", "Shipped", "Delivered").

These tables are connected via the CustomerID in the Orders table, which acts as a foreign key referencing the CustomerID in the Customers table. This setup allows you to track which customer placed which order and to perform detailed analysis and reporting on customer behavior and order details.

### Let's code!

In [53]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool

import textwrap

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')

# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", textwrap.dedent("""
        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to generate a correct SQL query for a natural language question, given the description of the datasets.

        INSTRUCTION:
        1. First, analyze the datasets and the question, and lay out the steps you need to do to get to the answer.
        2. Analyze what preprocessing steps you have to accomplish before your SQL query becomes a simple operation.
        3. Generate the SQL query to answer the natural language question.
        4. Write out the complete query.
        5. Use a Reasoning and Acting approach, so make sure first to state your thoughts, before taking an action.

        CONSTRAINTS:
        1. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
        2. You can only use the columns in the tables that are given, or that you have created as a derivative from those.
        3. Make sure the SQL query can run using SQLITE3
                               

        Here are the descriptions of two datasets that I have access to:
        '''
        ### Table 1: Customers
        This table stores information about the customers who purchase items from the e-commerce store.

        Columns:
        - CustomerID (INT): A unique identifier for each customer.
        - FirstName (VARCHAR): The first name of the customer.
        - LastName (VARCHAR): The last name of the customer.
        - Email (VARCHAR): The email address of the customer.
        - PhoneNumber (VARCHAR, optional): The phone number of the customer, which can be null if not provided.
        - CreateDate (DATETIME): The date and time when the customer was added to the database.

        ### Table 2: Orders
        This table records details about orders placed by customers.

        Columns:
        - OrderID (INT): A unique identifier for each order.
        - CustomerID (INT): A reference to CustomerID from the Customers table, linking each order to a specific customer.
        - OrderDate (DATETIME): The date and time when the order was placed.
        - ShippingAddress (VARCHAR): The address where the order is to be shipped.
        - TotalAmount (DECIMAL): The total amount of the order in dollars.
        - OrderStatus (VARCHAR): The status of the order (e.g., "Pending", "Shipped", "Delivered").                           
        '''
                                   
        Here is the natural language question:
        Construct a SQL query to identify customers who placed orders in their first month but did not place any orders in the subsequent month. The query should return the CustomerID, first name, last name, and the date of their last order. Additionally, provide a count of how many orders were placed in the first month of their activity.

        Begin!
        {agent_scratchpad}
        """)),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | chat_llm                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[], verbose=True)

response = agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: {'intermediate_steps': []}
### Reasoning and Plan

To solve this problem, we need to follow several steps:

1. **Identify the First Month of Activity for Each Customer**:
    - Calculate the first month each customer placed an order by extracting the month and year from the `OrderDate`.

2. **Count Orders in the First Month**:
    - For each customer, count the number of orders placed in their first month of activity.

3. **Identify Orders in the Subsequent Month**:
    - Check if there are any orders in the month following their first order month.

4. **Filter Customers**:
    - Filter out customers who placed orders in the subsequent month.

5. **Retrieve Required Customer Information**:
    - For the customers who did not place any orders in the subsequent month, retrieve their `CustomerID`, `FirstName`, `LastName`, and the date of their last order.

### SQL Query

To achieve this, we can break down the query into smaller parts and the

In [50]:
print(response['output'])

### Analysis and Steps

#### Step 1: Understand the Question
We need to list each customer along with their most recent order details. The required fields are:
- Customer's first name
- Customer's last name
- OrderID of the most recent order
- OrderDate of the most recent order
- TotalAmount of the most recent order
- OrderStatus of the most recent order
- Number of days elapsed since the most recent order

Finally, the results should be sorted by the elapsed days in ascending order.

#### Step 2: Identify Relevant Tables and Columns
We will need to use both the `Customers` and `Orders` tables. The columns we need are:
- From `Customers`: CustomerID, FirstName, LastName
- From `Orders`: OrderID, CustomerID, OrderDate, TotalAmount, OrderStatus

#### Step 3: Steps to Formulate the SQL Query
1. Join the `Customers` and `Orders` tables using the `CustomerID`.
2. For each customer, identify the most recent order. This can be done by finding the maximum `OrderDate` for each customer.
3. Calc

## Run your queries

In [54]:
import sqlite3
import pandas as pd

# Load data from CSV files into pandas DataFrames
customers_df = pd.read_csv('data/customers.csv')
orders_df = pd.read_csv('data/orders.csv')

# Create a connection to an SQLite in-memory database
conn = sqlite3.connect(':memory:')

# Transfer data from pandas DataFrames to SQLite
customers_df.to_sql('customers', conn, index=False, if_exists='replace')
orders_df.to_sql('orders', conn, index=False, if_exists='replace')

# Function to execute SQL queries and return the results as a DataFrame
def execute_query(query):
    return pd.read_sql_query(query, conn)

# Example SQL query to fetch data
query = """
SELECT
    C.CustomerID,
    C.FirstName,
    C.LastName,
    MAX(O.OrderDate) AS LastOrderDate,
    FMO.FirstMonthOrderCount
FROM Customers C
JOIN Orders O ON C.CustomerID = O.CustomerID
JOIN (
    SELECT
        O.CustomerID,
        COUNT(O.OrderID) AS FirstMonthOrderCount
    FROM Orders O
    JOIN (
        SELECT
            CustomerID,
            MIN(strftime('%Y-%m', OrderDate)) AS FirstOrderMonth
        FROM Orders
        GROUP BY CustomerID
    ) FOM ON O.CustomerID = FOM.CustomerID AND strftime('%Y-%m', O.OrderDate) = FOM.FirstOrderMonth
    GROUP BY O.CustomerID
) FMO ON C.CustomerID = FMO.CustomerID
LEFT JOIN (
    SELECT
        O.CustomerID,
        COUNT(O.OrderID) AS SubsequentMonthOrderCount
    FROM Orders O
    JOIN (
        SELECT
            CustomerID,
            MIN(strftime('%Y-%m', OrderDate)) AS FirstOrderMonth
        FROM Orders
        GROUP BY CustomerID
    ) FOM ON O.CustomerID = FOM.CustomerID AND strftime('%Y-%m', O.OrderDate) = strftime('%Y-%m', date(FOM.FirstOrderMonth || '-01', '+1 month'))
    GROUP BY O.CustomerID
) SMO ON C.CustomerID = SMO.CustomerID
WHERE SMO.SubsequentMonthOrderCount IS NULL
GROUP BY C.CustomerID, C.FirstName, C.LastName, FMO.FirstMonthOrderCount
"""

# Execute the query
result = execute_query(query)

# Close the database connection
conn.close()

In [55]:
result

,CustomerID,FirstName,LastName,LastOrderDate,FirstMonthOrderCount
0,1,Christopher,Yoder,2024-04-25 09:08:27,2
1,2,Robin,Williams,2024-06-06 10:49:36,1
2,3,Madeline,Hammond,2024-05-23 01:37:43,2
3,4,Mark,Robertson,2024-03-09 15:05:06,1
4,6,Michael,Hoffman,2024-05-23 06:38:55,2
5,7,Shawn,Smith,2024-06-05 21:27:43,1
6,12,Katherine,Christensen,2024-06-19 06:06:12,1
7,13,Heidi,Spencer,2024-06-21 07:22:22,1
8,16,Tiffany,Hudson,2024-02-09 07:38:58,1
9,17,Anita,Walls,2024-05-23 07:27:55,1
